In [1]:
CORE_CHANNELS = ["instagram", "naver_blog", "naver_place", "kakao_channel"]
AUX_CHANNELS  = ["youtube_shorts", "tiktok", "influencer_brief"]

def decide_channels(preferred_channels: list[str] | None, hinted_channels: list[str] | None) -> list[str]:
    """
    - preferred_channels가 있으면 그것만 반환 (교집합이든 아니든 그대로)
    - 없으면 hinted_channels에서 CORE 우선 + AUX는 브리프 전용으로 추가
    - 아무것도 없으면 CORE 기본 세트
    """
    if preferred_channels:
        return [c for c in preferred_channels if c in CORE_CHANNELS + AUX_CHANNELS] or CORE_CHANNELS
    hinted_channels = hinted_channels or []
    picked = [c for c in hinted_channels if c in CORE_CHANNELS]
    if not picked:
        picked = CORE_CHANNELS[:]  # 기본
    for c in hinted_channels:
        if c in AUX_CHANNELS and c not in picked:
            picked.append(c)
    return picked

In [2]:
import re

def parse_team2_to_brief(team2: dict) -> dict:
    blob = " ".join([
        " ".join(m.get("content","") for m in team2.get("messages",[])),
        team2.get("stp_strategy_document","") or "",
        team2.get("customer_personas","") or "",
        team2.get("targeting_proposal","") or "",
        team2.get("positioning_statement","") or "",
        team2.get("market_opportunities","") or ""
    ])

    industry_candidates = ["카페","치킨","한식","베이커리","디저트","피자","버거","분식","이자카야","파스타"]
    industry = next((w for w in industry_candidates if w in blob), None)
    if industry in ["베이커리","디저트"]:
        industry = "카페"
    if not industry:
        industry = "카페"

    if re.search(r"학생|대학|20대", blob): persona = "학생"
    elif re.search(r"직장|퇴근|오피스|직장인|30대", blob): persona = "직장인"
    elif re.search(r"가족|아이|주말\s*가족", blob): persona = "가족"
    else: persona = "직장인"

    goal = "재방문율 상승" if "재방문" in blob else "신규 유입 증가"

    tone = None
    if re.search(r"프리미엄|미식|특별|고급|레스토랑", blob):
        tone = "premium"
    elif re.search(r"미니멀|깔끔|심플", blob):
        tone = "minimal"

    season = None
    preferred_channels = None

    return {
        "industry": industry,
        "persona": persona,
        "goal": goal,
        "tone": tone,
        "season": season,
        "preferred_channels": preferred_channels
    }


In [22]:
"""
LLM(Gemini) → Query 6개 생성(페르소나→업종→톤→시즌 + 상권형용사)
→ Pexels에서 '쿼리당 최대 1장' 수집 (portrait, color=brown 우선)
→ 부족 시 컬러 완화/중복 제거/복제 보정으로 항상 6장
→ 2×3 HTML 무드보드 렌더 (topic + color chip + 링크)
"""

import os, re, json, random, requests
from typing import List, Optional

# =========================
# 0) 옵션: dominant color 추출(없으면 avg_color 사용)
# =========================
_DOM_COLOR_ON = False  # ← 지배색 사용하려면 True로
try:
    if _DOM_COLOR_ON:
        from PIL import Image
        import numpy as np
        from sklearn.cluster import KMeans
        import io
        _DOM_OK = True
    else:
        _DOM_OK = False
except Exception:
    _DOM_OK = False

def _dominant_hex(image_url: str) -> Optional[str]:
    if not _DOM_OK:
        return None
    try:
        r = requests.get(image_url, timeout=12)
        r.raise_for_status()
        im = Image.open(io.BytesIO(r.content)).convert("RGB")
        im = im.resize((160, 160))
        arr = np.array(im).reshape(-1, 3).astype(float)
        km = KMeans(n_clusters=3, n_init=4, random_state=42).fit(arr)
        counts = np.bincount(km.labels_)
        center = km.cluster_centers_[counts.argmax()]
        rgb = [max(0, min(255, int(round(c)))) for c in center]
        return "#{:02x}{:02x}{:02x}".format(*rgb)
    except Exception:
        return None

# =========================
# 1) 템플릿 (LLM 힌트용)
# =========================
INDUSTRY_TEMPLATE = {
    "카페": {"keywords": ["coffee shop interior","latte art","dessert flatlay","coffee bar","cake display"]},
    "치킨": {"keywords": ["fried chicken","beer combo","restaurant interior","crispy chicken"]},
    "한식": {"keywords": ["korean restaurant interior","banchan table","stone pot","modern hanok dining"]},
}
PERSONA_TEMPLATE = {
    "학생":   ["study","notebook","night","cozy","desk"],
    "직장인": ["lunch","office break","minimal","clean","after work"],
    "가족":   ["sharing","weekend","bright","table food","kids friendly"],
}

# --- 사람/장면 필터 ---
PEOPLE_BLOCKLIST = [
    "person","people","portrait","model","woman","man","lady","gentleman",
    "dancer","kids","children","child","selfie","pose","fashion","outfit",
    "headshot","face","smile","handsome","beautiful","bride","groom"
]
SCENE_WHITELIST = [
    "interior","table","counter","bar","kitchen","menu","flatlay","plating",
    "latte","espresso","dessert","dish","cup","mug","banchan","storefront",
    "tray","utensils","cutlery","place setting","booth","seating","cafeteria"
]

def _looks_like_people(alt: str) -> bool:
    t = (alt or "").lower()
    return any(w in t for w in PEOPLE_BLOCKLIST)

def _looks_like_scene(alt: str) -> bool:
    t = (alt or "").lower()
    return any(w in t for w in SCENE_WHITELIST)

# =========================
# 2) LLM: Gemini로 쿼리 6개 생성 (자동 모델 탐색 + 폴백)
# =========================
def _pick_available_model() -> str:
    import google.generativeai as genai
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
    try:
        models = list(genai.list_models())
        cand = [m.name for m in models if "generateContent" in getattr(m, "supported_generation_methods", [])]
    except Exception:
        cand = []
    priority = [
        "1.5-pro-latest","1.5-pro-002","1.5-pro",
        "1.5-flash-latest","1.5-flash-002","1.5-flash",
        "1.0-pro","gemini-pro"
    ]
    for p in priority:
        for name in cand:
            if p in name:
                return name.replace("models/", "")
    return "gemini-1.5-flash"

def _call_gemini_json_array(prompt: str, n: int) -> Optional[List[str]]:
    import google.generativeai as genai
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
    candidates = []
    if os.getenv("GEMINI_MODEL"):
        candidates.append(os.getenv("GEMINI_MODEL"))
    candidates += [_pick_available_model(), "gemini-1.5-flash", "gemini-pro"]
    tried = set()
    for name in candidates:
        if not name or name in tried:
            continue
        tried.add(name)
        try:
            model = genai.GenerativeModel(name)
            resp = model.generate_content(prompt)
            text = (resp.text or "").strip()
            m = re.search(r"\[.*\]", text, re.S)
            if not m:
                continue
            arr = json.loads(m.group(0))
            out, seen = [], set()
            for q in arr:
                q = re.sub(r"\s+", " ", str(q).strip())
                if q and q not in seen:
                    seen.add(q); out.append(q)
                if len(out) >= n:
                    break
            if out:
                return out
        except Exception:
            continue
    return None

def build_queries_with_gemini(
    industry: str,
    persona: str,
    tone: Optional[str] = None,
    season: Optional[str] = None,
    market_traits: Optional[List[str]] = None,   # ← 상권 형용사(예: ["trendy","youthful"])
    n: int = 6
) -> List[str]:
    ind_block = INDUSTRY_TEMPLATE.get(industry, {"keywords": []})
    per_block = PERSONA_TEMPLATE.get(persona, [])

    traits_line = ""
    if market_traits:
        traits_line = f"- 상권/지명은 쓰지 말고, 다음 분위기 힌트를 '형용사'로 자연스럽게 반영: {', '.join(market_traits)}\n"

    prompt = f"""
다음 요구에 맞춰 이미지 검색용 쿼리 {n}개를 JSON 배열로만 출력하세요.
규칙:
1) '페르소나 → 업종 → 톤(선택) → 시즌(선택)' 의미가 드러나도록 간결한 영문 구.
2) 각 쿼리는 2~5단어(예: "study cafe interior", "latte art close-up").
3) 상권/지명(성수/왕십리 등) 금지. 저작권/브랜드/인명/로고/워터마크 금지.
- 각 쿼리는 다음 키워드 중 최소 1개를 포함: interior, table, counter, bar, kitchen, menu, flatlay, plating, latte, dessert, dish, cup, storefront, banchan
- 다음 단어는 포함 금지: person, people, portrait, model, woman, man, kid, child, selfie, pose, fashion, outfit, headshot, face
{traits_line}
입력:
- 업종: {industry}
- 업종 템플릿 키워드: {", ".join(ind_block.get("keywords", []))}
- 페르소나: {persona}
- 페르소나 템플릿: {", ".join(per_block)}
- 톤: {tone or "없음"}
- 시즌/날씨: {season or "없음"}
출력 예시: ["study cafe interior", "latte art close-up", "dessert flatlay", ...]
JSON만 출력.
""".strip()

    arr = _call_gemini_json_array(prompt, n)
    if arr and len(arr) >= n:
        return arr

    # 폴백(템플릿)
    if industry == "카페":
        base = ["study cafe interior","latte art close-up","dessert flatlay","coffee bar","cake display","minimal coffee shop"]
    elif industry == "치킨":
        base = ["fried chicken","beer combo","restaurant interior","crispy chicken close-up","wood table chicken","sharing platter"]
    elif industry == "한식":
        base = ["korean restaurant interior","banchan table","stone pot","modern hanok dining","table setup","side dishes close-up"]
    else:
        base = ["restaurant interior","signature dish","table setup","chef plating","close-up dish","bar counter"]
    return base[:n]

# =========================
# 3) Pexels: 쿼리당 최대 1장 + 항상 6장
# =========================
def _get_clean_pexels_key() -> str:
    raw = os.getenv("PEXELS_API_KEY", "")
    key = (raw or "").strip()  # 앞뒤 공백/개행 제거
    # Windows PowerShell에서 따옴표까지 들어간 경우 제거
    if (key.startswith('"') and key.endswith('"')) or (key.startswith("'") and key.endswith("'")):
        key = key[1:-1].strip()
    # ASCII 아닌 문자가 있으면 바로 에러(헤더 인코딩 불가)
    if not key:
        raise RuntimeError("PEXELS_API_KEY 환경변수가 비어 있습니다.")
    try:
        key.encode("latin-1")
    except UnicodeEncodeError:
        # 디버깅 도움: 어떤 문자가 섞였는지 표시
        bad = "".join(ch for ch in key if ord(ch) > 255)
        raise RuntimeError(
            f"PEXELS_API_KEY에 비ASCII 문자가 섞였습니다: {repr(bad)}\n"
            "Pexels 대시보드에서 발급받은 영문/숫자 키를 그대로 넣으세요. (한글/공백/개행/따옴표 제거)"
        )
    return key

def _pexels_search(query, page=1, per_page=30, color="brown", orientation="portrait"):
    api_key = _get_clean_pexels_key()  # ← 여기!
    url = (
        "https://api.pexels.com/v1/search"
        f"?query={query}&per_page={per_page}&page={page}&orientation={orientation}"
        + (f"&color={color}" if color else "")
    )
    r = requests.get(url, headers={"Authorization": api_key}, timeout=15)
    if r.status_code == 401:
        raise RuntimeError("Pexels 401 Unauthorized: 키가 잘못/만료/공백 포함일 수 있습니다.")
    r.raise_for_status()
    return r.json().get("photos", []) or []


def fetch_photos_exact_6(
    queries: List[str],
    orientation: str = "portrait",
    prefer_color: Optional[str] = "brown"
):
    """
    - 쿼리당 최대 1장 랜덤 선택(기획 준수)
    - color 우선 → 부족 시 color=None
    - ALT 기반으로 '사람' 차단 + '장면' 보증
    - 중복 제거 후, 부족하면 마지막 이미지 복제로 6장 보장
    """
    if not queries:
        queries = ["coffee shop interior","latte art close-up","dessert flatlay","coffee bar","cake display","minimal coffee shop"]
    queries = list(queries)
    random.shuffle(queries)

    results, seen = [], set()

    for color in [prefer_color, None]:
        for q in queries:
            if len(results) >= 6:
                break
            picked = False
            # 페이지 탐색(최대 2페이지)
            for page in (1, 2):
                photos = _pexels_search(q, page=page, per_page=30, color=color, orientation=orientation)
                random.shuffle(photos)  # 랜덤 선별
                for p in photos:
                    src = p.get("src", {}) or {}
                    url = src.get("portrait") or src.get("large") or src.get("large2x") or p.get("url")
                    alt = p.get("alt") or ""
                    if not url or url in seen:
                        continue
                    # 1) 사람 사진 차단
                    if _looks_like_people(alt):
                        continue
                    # 2) 장면 보증
                    if not _looks_like_scene(alt):
                        continue

                    seen.add(url)
                    # 색상칩: dominant 우선(옵션), 없으면 avg_color
                    dom = _dominant_hex(url) if _DOM_OK else None
                    color_hex = dom or p.get("avg_color", "#999999")
                    results.append({
                        "img": url,
                        "alt": alt or "Moodboard photo",
                        "photographer": p.get("photographer", "Unknown"),
                        "photographer_url": p.get("photographer_url", "#"),
                        "page_url": p.get("url", "#"),
                        "avg_color": color_hex,
                    })
                    picked = True
                    break
                if picked or len(results) >= 6:
                    break
        if len(results) >= 6:
            break

    # 항상 6장 보정
    while results and len(results) < 6:
        results.append(results[-1].copy())
    return results[:6]

# =========================
# 4) 무드보드 HTML 생성/렌더
# =========================
def make_moodboard_html(items, title="Store Moodboard — 6 photos") -> str:
    def make_card(p, idx):
        img = p["img"]; ph = p["photographer"]; ph_url = p["photographer_url"]
        url = p["page_url"]; topic = p["alt"]; color = p["avg_color"]
        return f"""
        <article class="card">
          <div class="thumb">
            <img src="{img}" alt="{topic}">
            <span class="badge">{idx:02d}</span>
          </div>
          <div class="meta">
            <div class="topic" title="{topic}">{topic}</div>
            <div class="row">
              <div class="swatch" style="background:{color}"></div>
              <span class="hex">{color}</span>
              <span class="spacer"></span>
              <a href="{ph_url}" target="_blank" rel="noreferrer">{ph}</a>
              <a href="{url}" target="_blank" rel="noreferrer" class="view">View</a>
            </div>
          </div>
        </article>
        """
    cards_html = "\n".join(make_card(p, i+1) for i, p in enumerate(items))
    return f"""
    <div class="wrap">
      <header class="header">
        <strong>{title}</strong>
        <div class="sub">6 photos · Provided by <a href="https://www.pexels.com" target="_blank" rel="noreferrer">Pexels</a></div>
      </header>
      <main class="grid">{cards_html}</main>
    </div>
    <style>
      .wrap {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; color:#111; }}
      .header {{ position: sticky; top:0; z-index:10; background:#fff; border-bottom:1px solid #eee; padding:10px 16px; }}
      .header .sub {{ font-size:12px; color:#666; }}
      .grid {{ display:grid; grid-template-columns: repeat(3, 1fr); gap: 16px; padding: 16px; max-width: 1200px; margin: 0 auto; }}
      @media (max-width: 900px) {{ .grid {{ grid-template-columns: repeat(2, 1fr); }} }}
      @media (max-width: 600px) {{ .grid {{ grid-template-columns: 1fr; }} }}
      .card {{ border:1px solid #eee; border-radius:12px; overflow:hidden; background:#fff; box-shadow:0 2px 10px rgba(0,0,0,.05); display:flex; flex-direction:column; }}
      .thumb {{ position:relative; background:#f6f6f6; }}
      .thumb img {{ display:block; width:100%; height:auto; }}
      .badge {{ position:absolute; top:10px; left:10px; background:rgba(0,0,0,.65); color:#fff; font-size:12px; padding:4px 8px; border-radius:999px; }}
      .meta {{ padding:10px; display:flex; flex-direction:column; gap:8px; }}
      .topic {{ font-size:13px; line-height:1.3; color:#222; display:-webkit-box; -webkit-line-clamp:2; -webkit-box-orient:vertical; overflow:hidden; }}
      .row {{ display:flex; align-items:center; gap:8px; font-size:12px; color:#555; }}
      .swatch {{ width:12px; height:12px; border-radius:50%; border:1px solid rgba(0,0,0,.1); }}
      .hex {{ color:#666; }}
      .spacer {{ flex:1; }}
      .view {{ margin-left:8px; color:#0b57d0; text-decoration:none; font-weight:600; }}
      .view:hover {{ text-decoration:underline; }}
    </style>
    """

# (주피터 전용 미리보기용 — 일반 .py 실행에선 사용 안 함)
def render_moodboard_html(items, title="Store Moodboard — 6 photos"):
    from IPython.display import HTML, display
    display(HTML(make_moodboard_html(items, title=title)))


In [4]:
def build_moodboard_from_brief(brief: dict) -> tuple[list[str], list[dict], str]:
    """
    brief에서 쿼리 6개 → 사진 6장 → HTML
    return: (queries, photos6, html)
    """
    queries = build_queries_with_gemini(
        industry=brief["industry"],
        persona=brief["persona"],
        tone=brief.get("tone"),
        season=brief.get("season"),
        market_traits=brief.get("market_traits"),  # 없으면 None
        n=6
    )
    photos6 = fetch_photos_exact_6(
        queries=queries,
        orientation="portrait",
        prefer_color="brown"
    )
    title = f"Store Moodboard — {brief['persona']}×{brief['industry']}"
    html = make_moodboard_html(photos6, title=title)
    return queries, photos6, html



In [23]:
def build_concepts_with_llm(brief: dict, n: int = 3) -> list[dict]:
    """
    각 콘셉트 = {title, hook, message, why[]}
    """
    import google.generativeai as genai, json, re, os
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

    persona = brief["persona"]; industry = brief["industry"]
    tone = brief.get("tone") or "neutral"; goal = brief.get("goal") or "브랜드 인지도 상승"

    prompt = f"""
다음 정보로 마케팅 콘텐츠 콘셉트 {n}개를 JSON 배열로 출력하세요.
규칙:
- 각 아이템: title(짧은 테마명), hook(한 줄 훅), message(핵심 메시지 1~2문장), why(근거 리스트 2~3개)
- 업종/페르소나/톤/목표 반영, 이모지 1~2개 허용
입력:
- 업종: {industry}
- 페르소나: {persona}
- 톤: {tone}
- 목표: {goal}
출력 예시:
[
  {{"title":"시험주 달달 세트","hook":"밤샘 공부엔 당충전 ☕🍰","message":"달콤한 디저트와 진한 라떼로 집중력을 챙기세요.","why":["시험기간 방문↑","당충전 니즈"]}},
  {{"title":"주말 감성 브런치","hook":"주말엔 느긋하게 한 잔","message":"편안한 인테리어와 브런치로 여유를.","why":["주말 체류시간↑","감성 선호"]}}
]
JSON만 출력.
""".strip()

    try:
        model = genai.GenerativeModel(os.getenv("GEMINI_MODEL","gemini-1.5-flash"))
        resp = model.generate_content(prompt)
        text = (resp.text or "").strip()
        m = re.search(r"\[.*\]", text, re.S)
        if m:
            arr = json.loads(m.group(0))
            # sanity trim
            out = []
            for it in arr[:n]:
                out.append({
                    "title": it.get("title","").strip()[:40] or "콘셉트",
                    "hook": it.get("hook","").strip()[:80],
                    "message": it.get("message","").strip()[:140],
                    "why": [w.strip() for w in it.get("why",[])][:3]
                })
            if out:
                return out
    except Exception:
        pass

    # 폴백
    fallback = [
        {"title":"퇴근 후 달콤 리추얼","hook":"오늘만큼은, 나를 위한 달콤함 🍰",
         "message":"부드러운 라떼와 디저트로 하루의 마침표를.", "why":["퇴근 후 보상심리","감성 선호"]},
        {"title":"주말 브런치 타임","hook":"느긋한 한 잔의 여유 ☕",
         "message":"여유로운 음악과 브런치로 재충전하세요.", "why":["주말 체류시간↑","여가/휴식 니즈"]},
        {"title":"프리미엄 테이크아웃","hook":"집에서도 레스토랑 퀄리티",
         "message":"패키징부터 맛까지 프리미엄 배달 경험.", "why":["프리미엄 배달 수요↑","가치소비 경향"]}
    ]
    return fallback[:n]

In [24]:
def build_channel_guides_with_llm(brief: dict, concepts: list[dict], channels: list[str], schedule_hint: dict | None = None) -> dict:
    """
    instagram/naver_blog/naver_place/kakao_channel: 실행 가이드
    youtube_shorts/tiktok/influencer_brief: 브리프
    schedule_hint가 있으면 upload_time/send_time 기본값으로 주입(LLM 반환값이 비었을 때 채움)
    """
    import google.generativeai as genai, json, re, os
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

    persona = brief["persona"]; industry = brief["industry"]
    tone = brief.get("tone") or "neutral"; goal = brief.get("goal") or "브랜드 인지도 상승"

    concepts_for_prompt = [
        {"title": c["title"], "hook": c["hook"], "message": c["message"]} for c in concepts
    ][:3]

    schema_hint = {
      "instagram": ["format","duration","cut_list","copy","hashtags","upload_time"],
      "naver_blog": ["title","outline","cover_suggestion","copy","cta"],
      "naver_place": ["keywords","owner_reply_template","event_copy"],
      "kakao_channel": ["message","button_text","send_time"],
      "youtube_shorts": ["concept","hook","script_beats","hashtags"],
      "tiktok": ["concept","hook","script_beats","hashtags"],
      "influencer_brief": ["angle","deliverables","hashtags","notes"]
    }

    prompt = f"""
다음 정보로 채널별 콘텐츠 가이드를 JSON으로 출력하세요.
규칙:
- instagram/naver_blog/naver_place/kakao_channel: 실제 게시 가능하도록 세부 항목 포함
- youtube_shorts/tiktok/influencer_brief: 콘셉트/스크립트/해시태그 중심 브리프
- 해시태그는 8~15개 (국문/영문 혼합 가능, 업종/페르소나 맥락 반영)
입력:
- 업종: {industry}
- 페르소나: {persona}
- 톤: {tone}
- 목표: {goal}
- 채널: {channels}
- 콘셉트(요약): {json.dumps(concepts_for_prompt, ensure_ascii=False)}
출력 스키마 힌트: {json.dumps(schema_hint, ensure_ascii=False)}
JSON만 출력.
""".strip()

    out = {}
    try:
        model = genai.GenerativeModel(os.getenv("GEMINI_MODEL","gemini-1.5-flash"))
        resp = model.generate_content(prompt)
        text = (resp.text or "").strip()
        m = re.search(r"\{.*\}", text, re.S)
        if m:
            raw = json.loads(m.group(0))
            out = {k: raw.get(k) for k in channels if raw.get(k)}
    except Exception:
        out = {}

    # 폴백/보강
    if "instagram" in channels and "instagram" not in out:
        out["instagram"] = {
            "format":"reels","duration":"10~12s",
            "cut_list":["외관 인서트","라떼 아트 클로즈업","디저트 플랫레이","좌석/조명 무드샷","로고/간판"],
            "copy":"오늘만큼은, 나를 위한 달콤한 휴식 🍰☕",
            "hashtags":["#성수카페","#라떼아트","#디저트맛집","#퇴근후한잔","#미식경험","#브런치","#카공카페"],
        }
    if "naver_blog" in channels and "naver_blog" not in out:
        out["naver_blog"] = {
            "title":"시험기간 카공하기 좋은 감성 카페, 디저트 추천",
            "outline":["분위기/좌석","대표 디저트","라떼 아트","이벤트 안내","위치/영업시간"],
            "cover_suggestion":"라떼 아트 클로즈업 or 디저트 플랫레이",
            "copy":"퇴근 후 보상 심리 딱 맞는 달콤한 한 잔.",
            "cta":"리뷰 작성 시 리필 쿠폰 증정"
        }
    if "kakao_channel" in channels and "kakao_channel" not in out:
        out["kakao_channel"] = {
            "message":"오늘 저녁, 나를 위한 달콤한 보상 어떠세요? 🍰",
            "button_text":"메뉴 보기",
        }
    if "naver_place" in channels and "naver_place" not in out:
        out["naver_place"] = {
            "keywords":["성수동 카페","라떼아트","디저트 맛집","카공"],
            "owner_reply_template":"방문 감사합니다 :) 더 좋은 시간 되실 수 있게 노력하겠습니다!",
            "event_copy":"리뷰 작성 시 아메리카노 리필"
        }

    # 스케줄 힌트 주입
    if schedule_hint:
        # instagram upload_time
        insta = out.get("instagram", {})
        if "upload_time" not in insta or not insta.get("upload_time"):
            hint = next((r["time"] for r in schedule_hint.get("recommendations", [])
                         if r.get("channel")=="instagram"), None)
            if hint: insta["upload_time"] = hint
        if insta: out["instagram"] = insta

        # kakao_channel send_time
        kakao = out.get("kakao_channel", {})
        if "send_time" not in kakao or not kakao.get("send_time"):
            hint = next((r["time"] for r in schedule_hint.get("recommendations", [])
                         if r.get("channel")=="kakao_channel"), None)
            if hint: kakao["send_time"] = hint
        if kakao: out["kakao_channel"] = kakao

    return out


In [25]:
def assemble_final_package(store_name: str,
                           brief: dict,
                           queries: list[str],
                           photos6: list[dict],
                           concepts: list[dict],
                           channel_guides: dict,
                           evidence: list[str] | None = None,
                           schedule: dict | None = None) -> dict:
    return {
        "store": store_name,
        "target": f"{brief['persona']}",
        "persona": brief["persona"],
        "goal": brief.get("goal",""),
        "moodboard": [
            {"topic": p.get("alt","").strip()[:60] or "mood", "url": p["img"], "color": p.get("avg_color","#999999")}
            for p in photos6
        ],
        "queries": queries,
        "concepts": concepts,
        "channel_guides": channel_guides,
        "evidence": evidence or [],
        "schedule": schedule or {}
    }

In [26]:
 def run_content_creator_pipeline(team2_json: dict,
                                 preferred_channels: list[str] | None = None,
                                 hinted_channels: list[str] | None = None,
                                 store_name: str = "OO카페") -> dict:
    # (A) 근거 먼저 뽑기 → 스케줄 힌트 유도
    evidence = simple_evidence_from_team2(team2_json)
    schedule_hint = infer_schedule_from_evidence(evidence)

    # (B) 브리프 생성
    brief = parse_team2_to_brief(team2_json)

    # (C) 채널 결정
    channels = decide_channels(preferred_channels, hinted_channels)

    # (D) 무드보드
    #  - 이미 구현해둔 함수 사용 (LLM 쿼리 → Pexels 6장 → HTML)
    queries, photos6, _html = build_moodboard_from_brief(brief)

    # (E) 콘셉트
    concepts = build_concepts_with_llm(brief, n=3)

    # (F) 채널 가이드 (스케줄 힌트 반영)
    guides = build_channel_guides_with_llm(brief, concepts, channels, schedule_hint=schedule_hint)

    # (G) 최종 패키지
    final = assemble_final_package(
        store_name=store_name,
        brief=brief,
        queries=queries,
        photos6=photos6,
        concepts=concepts,
        channel_guides=guides,
        evidence=evidence,
        schedule=schedule_hint
    )
    return final


In [27]:
# streamlit_app.py
import os
import json
import streamlit as st
from typing import List

# === 에이전트 모듈(이미 구현되어 있다고 가정) ===
from ai_agent import (
    parse_team2_to_brief,         # Team2 JSON -> brief 추출
    build_moodboard_from_brief,   # brief -> (queries, photos6, html)
    run_content_creator_pipeline, # 전체 파이프라인 -> 최종 JSON
    make_moodboard_html           # (옵션) moodboard items -> HTML
)

st.set_page_config(page_title="Content Creator — Moodboard & Channel Guides", layout="wide")

# ---- 공용: 복사 버튼 컴포넌트 (textarea + Copy) ----
def copy_to_clipboard(label: str, text: str, height: int = 180, key: str = "copy1"):
    safe = text.replace("</", "<\\/")  # </script> 이슈 방지
    ta_id = f"copyArea-{key}"
    html = f"""
    <div style="font-family:system-ui, -apple-system, Segoe UI, Roboto, sans-serif;">
      <label style="font-weight:600;margin-bottom:6px;display:block;">{label}</label>
      <textarea id="{ta_id}" style="width:100%;height:{height}px;border:1px solid #ccc;border-radius:8px;padding:10px;">{safe}</textarea>
      <button onclick="navigator.clipboard.writeText(document.getElementById('{ta_id}').value)"
              style="margin-top:8px;padding:6px 10px;border:1px solid #0b57d0;border-radius:6px;background:#0b57d0;color:#fff;cursor:pointer;">
        📋 Copy to clipboard
      </button>
    </div>
    """
    st.components.v1.html(html, height=height+80)


# ---- 사이드바: 설정 & 환경 체크 ----
with st.sidebar:
    st.header("⚙️ 설정")
    st.caption("환경변수 준비: PEXELS_API_KEY, GEMINI_API_KEY")
    pexels_ok = bool(os.getenv("PEXELS_API_KEY"))
    gemini_ok = bool(os.getenv("GEMINI_API_KEY"))
    st.write("Pexels:", "✅" if pexels_ok else "❌")
    st.write("Gemini:", "✅" if gemini_ok else "❌")

    CORE_CHANNELS = ["instagram", "naver_blog", "naver_place", "kakao_channel"]
    AUX_CHANNELS  = ["youtube_shorts", "tiktok", "influencer_brief"]
    ALL_CHANNELS  = CORE_CHANNELS + AUX_CHANNELS

    preferred_channels = st.multiselect(
        "Preferred channels (선호 채널)",
        options=ALL_CHANNELS,
        default=["instagram","naver_blog","kakao_channel"]
    )

    hinted_channels = st.multiselect(
        "Hinted channels (질문/요청에 포함된 채널, 선택)",
        options=ALL_CHANNELS,
        default=[]
    )

    st.divider()
    st.caption("상권 분위기(형용사, 지명 X): 예) trendy, youthful")
    traits_str = st.text_input("Market traits (comma-separated)", "trendy,youthful")

# ---- 본문: 입력 JSON & 실행 ----
st.title("🧑‍🎨 Content Creator · Moodboard + Channel Guides")

DEFAULT_TEAM2 = {
  "messages": [
    {"content": "[Team 1 종합 진단 보고서] ... 재방문 저하/경쟁 강도 높음 ...", "name": "Supervisor"},
    {"content": "STP 전략 수립이 완료되었습니다. 최종 보고서를 확인해주세요.", "name": "Supervisor"}
  ],
  "stp_strategy_document": "# STP 전략 보고서\n\n... '프리미엄 배달' ... '20대 여성' ...",
  "market_opportunities": "- **기회:** 프리미엄 배달 수요 증가 ...",
  "customer_personas": "김지은 26세, 2년차 직장인, 배달 앱 VIP, 퇴근 후 미식 경험 선호",
  "targeting_proposal": "- **추천 타겟:** '김지은(26세)' ...",
  "positioning_statement": "- **USP:** '평범한 하루를 특별하게 만드는 미식 배달 경험' ..."
}
team2_text = st.text_area(
    "Team2(STP) JSON 입력",
    value=json.dumps(DEFAULT_TEAM2, ensure_ascii=False, indent=2),
    height=250
)

colL, colR = st.columns([1,1])
with colL:
    store_name = st.text_input("상호명", "OO카페")
with colR:
    st.caption("업종/페르소나/톤/시즌은 Team2에서 자동 추출되며, 필요시 이후 단계에서 수정 가능")

run_btn = st.button("생성하기", type="primary")

# ---- 실행 & 출력 ----
if run_btn:
    try:
        team2_json = json.loads(team2_text)

        # 1) Team2 -> brief
        brief = parse_team2_to_brief(team2_json)
        traits = [t.strip() for t in traits_str.split(",") if t.strip()]
        if traits:
            brief["market_traits"] = traits

        st.subheader("1) 브리프(Brief)")
        st.json(brief, expanded=False)

        # 2) 무드보드 (쿼리 6개 + 사진 6장 + HTML)
        st.subheader("2) 무드보드 (2×3)")
        queries, photos6, mood_html = build_moodboard_from_brief(brief)

        with st.expander("사용된 쿼리 6개 보기", expanded=False):
            st.write(queries)
            # 복사 버튼
            copy_to_clipboard("Queries (copy)", "\n".join(queries), height=120)

        # 무드보드 HTML 렌더
        st.components.v1.html(mood_html, height=900, scrolling=True)

        # 3) 전체 파이프라인 (채널 가이드 / 콘셉트 / 근거 포함 최종 JSON)
        st.subheader("3) 채널 가이드 & 최종 패키지")
        final = run_content_creator_pipeline(
            team2_json=team2_json,
            preferred_channels=preferred_channels,
            hinted_channels=hinted_channels,
            store_name=store_name
        )

        # 최종 JSON 복사 버튼 + 다운로드 버튼
        final_pretty = json.dumps(final, ensure_ascii=False, indent=2)
        copy_to_clipboard("Final JSON (copy)", final_pretty, height=280)
        st.download_button(
            "최종 JSON 다운로드",
            data=final_pretty,
            file_name=f"{store_name}_content_package.json",
            mime="application/json"
        )

        # 4) 채널 탭 UI
        guides = final.get("channel_guides", {}) or {}
        if guides:
            st.markdown("#### 채널별 가이드")
            tabs = st.tabs([c for c in guides.keys()])

            for tab, ch in zip(tabs, guides.keys()):
                with tab:
                    g = guides[ch]
                    st.write(f"**채널:** `{ch}`")
                    st.json(g, expanded=False)

                    # 채널별 하이라이트 (옵션)
                    if ch == "instagram":
                        st.write("**Format**:", g.get("format"))
                        st.write("**Duration**:", g.get("duration"))
                        st.write("**Cut list**:", g.get("cut_list"))
                        st.write("**Copy**:", g.get("copy"))
                        st.write("**Hashtags**:", ", ".join(g.get("hashtags", [])))
                        st.write("**Upload time**:", g.get("upload_time"))

                    elif ch == "naver_blog":
                        st.write("**Title**:", g.get("title"))
                        st.write("**Outline**:", g.get("outline"))
                        st.write("**Cover suggestion**:", g.get("cover_suggestion"))
                        st.write("**Copy**:", g.get("copy"))
                        st.write("**CTA**:", g.get("cta"))

                    elif ch == "naver_place":
                        st.write("**Keywords**:", ", ".join(g.get("keywords", [])))
                        st.write("**Owner reply template**:", g.get("owner_reply_template"))
                        st.write("**Event copy**:", g.get("event_copy"))

                    elif ch == "kakao_channel":
                        st.write("**Message**:", g.get("message"))
                        st.write("**Button**:", g.get("button_text"))
                        st.write("**Send time**:", g.get("send_time"))

                    elif ch in ("youtube_shorts","tiktok"):
                        st.write("**Concept**:", g.get("concept"))
                        st.write("**Hook**:", g.get("hook"))
                        st.write("**Script beats**:", g.get("script_beats"))
                        st.write("**Hashtags**:", ", ".join(g.get("hashtags", [])))

                    elif ch == "influencer_brief":
                        st.write("**Angle**:", g.get("angle"))
                        st.write("**Deliverables**:", g.get("deliverables"))
                        st.write("**Hashtags**:", ", ".join(g.get("hashtags", [])))
                        st.write("**Notes**:", g.get("notes"))

        # 5) 근거칩/스케줄 미리보기
        col1, col2 = st.columns([1,1])
        with col1:
            st.markdown("#### Evidence (근거칩)")
            st.write(final.get("evidence", []))
        with col2:
            st.markdown("#### Schedule (추천 업로드 시간)")
            st.json(final.get("schedule", {}), expanded=False)

    except Exception as e:
        st.error(f"오류가 발생했습니다: {e}")
        st.stop()

ModuleNotFoundError: No module named 'ai_agent'

In [28]:
from dotenv import load_dotenv
load_dotenv()  # 가장 위에서 호출

python-dotenv could not parse statement starting at line 3


True

In [30]:
# smoke_test_standalone.py
# -------------------------------------------------------------------
# Team2 JSON만으로 무드보드 + 채널 가이드까지 생성하는 단일 스크립트
# PEXELS_API_KEY 필수 / GEMINI_API_KEY 선택(없으면 폴백)
# 생성 결과:
#   - moodboard.html (2×3 그리드, 6컷)
#   - final_package.json (moodboard + concepts + channel_guides + evidence + schedule)
# -------------------------------------------------------------------
os.environ["PEXELS_API_KEY"] = "NiCSGOCv9sUFIyekjbTsVrp22ZDmTvTDHaFuAVUpsP3ENj6wWcHvIfP3"


import os, re, json, random, requests
from typing import List, Optional

# ========== 0) 옵션: dominant color 추출(없으면 avg_color 사용) ==========
_DOM_COLOR_ON = False  # 지배색 사용하려면 True로
try:
    if _DOM_COLOR_ON:
        from PIL import Image
        import numpy as np
        from sklearn.cluster import KMeans
        import io
        _DOM_OK = True
    else:
        _DOM_OK = False
except Exception:
    _DOM_OK = False

def _dominant_hex(image_url: str) -> Optional[str]:
    if not _DOM_OK:
        return None
    try:
        r = requests.get(image_url, timeout=12)
        r.raise_for_status()
        im = Image.open(io.BytesIO(r.content)).convert("RGB")
        im = im.resize((160, 160))
        arr = np.array(im).reshape(-1, 3).astype(float)
        km = KMeans(n_clusters=3, n_init=4, random_state=42).fit(arr)
        counts = np.bincount(km.labels_)
        center = km.cluster_centers_[counts.argmax()]
        rgb = [max(0, min(255, int(round(c)))) for c in center]
        return "#{:02x}{:02x}{:02x}".format(*rgb)
    except Exception:
        return None

# ========== 1) 템플릿 (LLM 힌트용) ==========
INDUSTRY_TEMPLATE = {
    "카페": {"keywords": ["coffee shop interior","latte art","dessert flatlay","coffee bar","cake display"]},
    "치킨": {"keywords": ["fried chicken","beer combo","restaurant interior","crispy chicken"]},
    "한식": {"keywords": ["korean restaurant interior","banchan table","stone pot","modern hanok dining"]},
}
PERSONA_TEMPLATE = {
    "학생":   ["study","notebook","night","cozy","desk"],
    "직장인": ["lunch","office break","minimal","clean","after work"],
    "가족":   ["sharing","weekend","bright","table food","kids friendly"],
}

# --- 사람/장면 필터 ---
PEOPLE_BLOCKLIST = [
    "person","people","portrait","model","woman","man","lady","gentleman",
    "dancer","kids","children","child","selfie","pose","fashion","outfit",
    "headshot","face","smile","handsome","beautiful","bride","groom"
]
SCENE_WHITELIST = [
    "interior","table","counter","bar","kitchen","menu","flatlay","plating",
    "latte","espresso","dessert","dish","cup","mug","banchan","storefront",
    "tray","utensils","cutlery","place setting","booth","seating","cafeteria"
]
def _looks_like_people(alt: str) -> bool:
    t = (alt or "").lower()
    return any(w in t for w in PEOPLE_BLOCKLIST)
def _looks_like_scene(alt: str) -> bool:
    t = (alt or "").lower()
    return any(w in t for w in SCENE_WHITELIST)

# ========== 2) Team2 → 브리프 추출 ==========
def parse_team2_to_brief(team2: dict) -> dict:
    blob = " ".join([
        " ".join(m.get("content","") for m in team2.get("messages",[])),
        team2.get("stp_strategy_document","") or "",
        team2.get("customer_personas","") or "",
        team2.get("targeting_proposal","") or "",
        team2.get("positioning_statement","") or "",
        team2.get("market_opportunities","") or ""
    ])

    industry_candidates = ["카페","치킨","한식","베이커리","디저트","피자","버거","분식","이자카야","파스타"]
    industry = next((w for w in industry_candidates if w in blob), None)
    if industry in ["베이커리","디저트"]: industry = "카페"
    if not industry: industry = "카페"

    if re.search(r"학생|대학|20대", blob): persona = "학생"
    elif re.search(r"직장|퇴근|오피스|직장인|30대", blob): persona = "직장인"
    elif re.search(r"가족|아이|주말\s*가족", blob): persona = "가족"
    else: persona = "직장인"

    goal = "재방문율 상승" if "재방문" in blob else "신규 유입 증가"

    tone = None
    if re.search(r"프리미엄|미식|특별|고급|레스토랑", blob):
        tone = "premium"
    elif re.search(r"미니멀|깔끔|심플", blob):
        tone = "minimal"

    season = None

    return {
        "industry": industry,
        "persona": persona,
        "goal": goal,
        "tone": tone,
        "season": season,
        "preferred_channels": None,
        # "market_traits": ["trendy","youthful"]  # 필요하면 외부에서 주입
    }

# ========== 3) LLM로 쿼리 6개 (없으면 폴백) ==========
def _pick_available_model() -> str:
    try:
        import google.generativeai as genai
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        models = list(genai.list_models())
        cand = [m.name for m in models if "generateContent" in getattr(m, "supported_generation_methods", [])]
    except Exception:
        cand = []
    priority = [
        "1.5-pro-latest","1.5-pro-002","1.5-pro",
        "1.5-flash-latest","1.5-flash-002","1.5-flash",
        "1.0-pro","gemini-pro"
    ]
    for p in priority:
        for name in cand:
            if p in name:
                return name.replace("models/","")
    return "gemini-1.5-flash"

def _call_gemini_json_array(prompt: str, n: int) -> Optional[List[str]]:
    try:
        import google.generativeai as genai
    except Exception:
        return None
    try:
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        candidates = []
        if os.getenv("GEMINI_MODEL"): candidates.append(os.getenv("GEMINI_MODEL"))
        candidates += [_pick_available_model(), "gemini-1.5-flash", "gemini-pro"]
        tried = set()
        for name in candidates:
            if not name or name in tried: continue
            tried.add(name)
            try:
                model = genai.GenerativeModel(name)
                resp = model.generate_content(prompt)
                text = (resp.text or "").strip()
                m = re.search(r"\[.*\]", text, re.S)
                if not m: 
                    continue
                arr = json.loads(m.group(0))
                out, seen = [], set()
                for q in arr:
                    q = re.sub(r"\s+", " ", str(q).strip())
                    if q and q not in seen:
                        seen.add(q); out.append(q)
                    if len(out) >= n: break
                if out:
                    return out
            except Exception:
                continue
    except Exception:
        return None
    return None

def build_queries_with_gemini(industry: str, persona: str,
                              tone: Optional[str]=None, season: Optional[str]=None,
                              market_traits: Optional[List[str]]=None, n: int=6) -> List[str]:
    ind_block = INDUSTRY_TEMPLATE.get(industry, {"keywords": []})
    per_block = PERSONA_TEMPLATE.get(persona, [])
    traits_line = ""
    if market_traits:
        traits_line = f"- 상권/지명 금지. 다음 분위기 힌트를 형용사로만 반영: {', '.join(market_traits)}\n"
    prompt = f"""
다음 요구에 맞춰 이미지 검색용 쿼리 {n}개를 JSON 배열로만 출력.
규칙:
1) '페르소나 → 업종 → 톤(선택) → 시즌(선택)' 의미가 드러나도록 2~5단어.
2) 상권/지명(성수/왕십리 등) 금지. 브랜드/인명/로고/워터마크 금지.
3) 다음 키워드 중 최소 1개 포함: interior, table, counter, bar, kitchen, menu, flatlay, plating, latte, dessert, dish, cup, storefront, banchan
4) 다음 단어 금지: person, people, portrait, model, woman, man, kid, child, selfie, pose, fashion, outfit, headshot, face
{traits_line}
입력:
- 업종: {industry}
- 업종 템플릿 키워드: {", ".join(ind_block.get("keywords", []))}
- 페르소나: {persona}
- 페르소나 템플릿: {", ".join(per_block)}
- 톤: {tone or "없음"}
- 시즌/날씨: {season or "없음"}
예시: ["study cafe interior","latte art close-up","dessert flatlay", ...]
JSON만 출력.
""".strip()

    arr = _call_gemini_json_array(prompt, n)
    if arr and len(arr) >= n:
        return arr

    # 폴백(템플릿)
    if industry == "카페":
        base = ["study cafe interior","latte art close-up","dessert flatlay","coffee bar","cake display","minimal coffee shop"]
    elif industry == "치킨":
        base = ["fried chicken","beer combo","restaurant interior","crispy chicken close-up","wood table chicken","sharing platter"]
    elif industry == "한식":
        base = ["korean restaurant interior","banchan table","stone pot","modern hanok dining","table setup","side dishes close-up"]
    else:
        base = ["restaurant interior","signature dish","table setup","chef plating","close-up dish","bar counter"]
    return base[:n]

# ========== 4) Pexels: 쿼리당 최대 1장 + 항상 6장 ==========
def _pexels_search(query, page=1, per_page=30, color="brown", orientation="portrait"):
    api_key = os.getenv("PEXELS_API_KEY")
    if not api_key:
        raise RuntimeError("PEXELS_API_KEY 환경변수가 비어 있습니다.")
    url = (
        "https://api.pexels.com/v1/search"
        f"?query={query}&per_page={per_page}&page={page}&orientation={orientation}"
        + (f"&color={color}" if color else "")
    )
    r = requests.get(url, headers={"Authorization": api_key}, timeout=15)
    if r.status_code == 401:
        raise RuntimeError("Pexels 401 Unauthorized: 키가 잘못/만료/공백 포함일 수 있습니다.")
    r.raise_for_status()
    return r.json().get("photos", []) or []

def fetch_photos_exact_6(queries: List[str], orientation: str="portrait", prefer_color: Optional[str]="brown"):
    if not queries:
        queries = ["coffee shop interior","latte art close-up","dessert flatlay","coffee bar","cake display","minimal coffee shop"]
    queries = list(queries)
    random.shuffle(queries)

    results, seen = [], set()
    for color in [prefer_color, None]:
        for q in queries:
            if len(results) >= 6: break
            picked = False
            for page in (1, 2):
                photos = _pexels_search(q, page=page, per_page=30, color=color, orientation=orientation)
                random.shuffle(photos)
                for p in photos:
                    src = p.get("src", {}) or {}
                    url = src.get("portrait") or src.get("large") or src.get("large2x") or p.get("url")
                    alt = p.get("alt") or ""
                    if not url or url in seen: continue
                    # 1) 사람 사진 차단
                    if _looks_like_people(alt): continue
                    # 2) 장면 보증
                    if not _looks_like_scene(alt): continue
                    seen.add(url)
                    dom = _dominant_hex(url) if _DOM_OK else None
                    color_hex = dom or p.get("avg_color", "#999999")
                    results.append({
                        "img": url,
                        "alt": alt or "Moodboard photo",
                        "photographer": p.get("photographer", "Unknown"),
                        "photographer_url": p.get("photographer_url", "#"),
                        "page_url": p.get("url", "#"),
                        "avg_color": color_hex,
                    })
                    picked = True
                    break
                if picked or len(results) >= 6:
                    break
        if len(results) >= 6:
            break

    while results and len(results) < 6:
        results.append(results[-1].copy())
    return results[:6]

# ========== 5) 무드보드 HTML ==========
def make_moodboard_html(items: List[dict], title="Store Moodboard — 6 photos") -> str:
    def make_card(p, idx):
        img = p["img"]; ph = p["photographer"]; ph_url = p["photographer_url"]
        url = p["page_url"]; topic = p["alt"]; color = p["avg_color"]
        return f"""
        <article class="card">
          <div class="thumb">
            <img src="{img}" alt="{topic}">
            <span class="badge">{idx:02d}</span>
          </div>
          <div class="meta">
            <div class="topic" title="{topic}">{topic}</div>
            <div class="row">
              <div class="swatch" style="background:{color}"></div>
              <span class="hex">{color}</span>
              <span class="spacer"></span>
              <a href="{ph_url}" target="_blank" rel="noreferrer">{ph}</a>
              <a href="{url}" target="_blank" rel="noreferrer" class="view">View</a>
            </div>
          </div>
        </article>
        """
    cards_html = "\n".join(make_card(p, i+1) for i, p in enumerate(items))
    return f"""
    <div class="wrap">
      <header class="header">
        <strong>{title}</strong>
        <div class="sub">6 photos · Provided by <a href="https://www.pexels.com" target="_blank" rel="noreferrer">Pexels</a></div>
      </header>
      <main class="grid">{cards_html}</main>
    </div>
    <style>
      .wrap {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; color:#111; }}
      .header {{ position: sticky; top:0; z-index:10; background:#fff; border-bottom:1px solid #eee; padding:10px 16px; }}
      .header .sub {{ font-size:12px; color:#666; }}
      .grid {{ display:grid; grid-template-columns: repeat(3, 1fr); gap: 16px; padding: 16px; max-width: 1200px; margin: 0 auto; }}
      @media (max-width: 900px) {{ .grid {{ grid-template-columns: repeat(2, 1fr); }} }}
      @media (max-width: 600px) {{ .grid {{ grid-template-columns: 1fr; }} }}
      .card {{ border:1px solid #eee; border-radius:12px; overflow:hidden; background:#fff; box-shadow:0 2px 10px rgba(0,0,0,.05); display:flex; flex-direction:column; }}
      .thumb {{ position:relative; background:#f6f6f6; }}
      .thumb img {{ display:block; width:100%; height:auto; }}
      .badge {{ position:absolute; top:10px; left:10px; background:rgba(0,0,0,.65); color:#fff; font-size:12px; padding:4px 8px; border-radius:999px; }}
      .meta {{ padding:10px; display:flex; flex-direction:column; gap:8px; }}
      .topic {{ font-size:13px; line-height:1.3; color:#222; display:-webkit-box; -webkit-line-clamp:2; -webkit-box-orient:vertical; overflow:hidden; }}
      .row {{ display:flex; align-items:center; gap:8px; font-size:12px; color:#555; }}
      .swatch {{ width:12px; height:12px; border-radius:50%; border:1px solid rgba(0,0,0,.1); }}
      .hex {{ color:#666; }}
      .spacer {{ flex:1; }}
      .view {{ margin-left:8px; color:#0b57d0; text-decoration:none; font-weight:600; }}
      .view:hover {{ text-decoration:underline; }}
    </style>
    """

def build_moodboard_from_brief(brief: dict):
    queries = build_queries_with_gemini(
        brief["industry"], brief["persona"],
        tone=brief.get("tone"), season=brief.get("season"),
        market_traits=brief.get("market_traits"), n=6
    )
    photos6 = fetch_photos_exact_6(queries, orientation="portrait", prefer_color="brown")
    html = make_moodboard_html(photos6, title=f"Store Moodboard — {brief['persona']}×{brief['industry']}")
    return queries, photos6, html

# ========== 6) 근거/스케줄 ==========
def simple_evidence_from_team2(team2: dict) -> List[str]:
    s = " ".join([team2.get("market_opportunities",""), team2.get("stp_strategy_document","")])
    e = []
    if "재방문" in s: e.append("repeat_rate 낮음 → 재방문 유도 필요")
    if "프리미엄 배달" in s: e.append("프리미엄 배달 수요↑ → 가치소비 타겟팅")
    if "경쟁" in s: e.append("상권 경쟁 강도↑ → 차별화 포지셔닝 필요")
    return e[:5]

def infer_schedule_from_evidence(evidence: List[str]) -> dict:
    # 간단 룰: '주말' 키워드가 있으면 주말 밤, 없으면 평일 저녁
    weekend = any("주말" in x for x in evidence)
    return {
        "recommendations": [
            {"channel":"instagram", "time": "금·토 21:00~23:00" if weekend else "평일 18:00~20:00"},
            {"channel":"kakao_channel", "time": "평일 17:30"}
        ]
    }

# ========== 7) 채널 선택 ==========
CORE_CHANNELS = ["instagram", "naver_blog", "naver_place", "kakao_channel"]
AUX_CHANNELS  = ["youtube_shorts", "tiktok", "influencer_brief"]
def decide_channels(preferred_channels: Optional[List[str]], hinted_channels: Optional[List[str]]) -> List[str]:
    if preferred_channels:
        return [c for c in preferred_channels if c in CORE_CHANNELS + AUX_CHANNELS] or CORE_CHANNELS
    hinted_channels = hinted_channels or []
    picked = [c for c in hinted_channels if c in CORE_CHANNELS]
    if not picked:
        picked = CORE_CHANNELS[:]
    for c in hinted_channels:
        if c in AUX_CHANNELS and c not in picked:
            picked.append(c)
    return picked

# ========== 8) 콘셉트 & 채널 가이드 (LLM or 폴백) ==========
def build_concepts_with_llm(brief: dict, n: int=3) -> List[dict]:
    persona = brief["persona"]; industry = brief["industry"]
    tone = brief.get("tone") or "neutral"; goal = brief.get("goal") or "브랜드 인지도 상승"
    prompt = f"""
다음 정보로 마케팅 콘텐츠 콘셉트 {n}개를 JSON 배열로 출력.
각 아이템: title, hook, message, why[]
업종:{industry} / 페르소나:{persona} / 톤:{tone} / 목표:{goal}
예시:[
  {{"title":"시험주 달달 세트","hook":"밤샘 공부엔 당충전 ☕🍰","message":"달콤한 디저트와 진한 라떼로 집중력.","why":["시험기간 방문↑","당충전 니즈"]}}
]
JSON만.
""".strip()
    try:
        import google.generativeai as genai
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        model = genai.GenerativeModel(os.getenv("GEMINI_MODEL","gemini-1.5-flash"))
        resp = model.generate_content(prompt)
        text = (resp.text or "").strip()
        m = re.search(r"\[.*\]", text, re.S)
        if m:
            arr = json.loads(m.group(0))[:n]
            out=[]
            for it in arr:
                out.append({
                    "title": it.get("title","").strip()[:40] or "콘셉트",
                    "hook": it.get("hook","").strip()[:80],
                    "message": it.get("message","").strip()[:140],
                    "why": [w.strip() for w in it.get("why",[])][:3]
                })
            if out: return out
    except Exception:
        pass
    # 폴백
    fallback = [
        {"title":"퇴근 후 달콤 리추얼","hook":"오늘만큼은, 나를 위한 달콤함 🍰",
         "message":"부드러운 라떼와 디저트로 하루의 마침표.", "why":["퇴근 후 보상심리","감성 선호"]},
        {"title":"주말 브런치 타임","hook":"느긋한 한 잔의 여유 ☕",
         "message":"여유로운 음악과 브런치로 재충전.", "why":["주말 체류시간↑","여가/휴식 니즈"]},
        {"title":"프리미엄 테이크아웃","hook":"집에서도 레스토랑 퀄리티",
         "message":"패키징부터 맛까지 프리미엄 배달 경험.", "why":["프리미엄 배달 수요↑","가치소비 경향"]}
    ]
    return fallback[:n]

def build_channel_guides_with_llm(brief: dict, concepts: List[dict], channels: List[str], schedule_hint: Optional[dict]=None) -> dict:
    persona = brief["persona"]; industry = brief["industry"]
    tone = brief.get("tone") or "neutral"; goal = brief.get("goal") or "브랜드 인지도 상승"
    concepts_for_prompt = [{"title":c["title"], "hook":c["hook"], "message":c["message"]} for c in concepts][:3]
    schema_hint = {
      "instagram": ["format","duration","cut_list","copy","hashtags","upload_time"],
      "naver_blog": ["title","outline","cover_suggestion","copy","cta"],
      "naver_place": ["keywords","owner_reply_template","event_copy"],
      "kakao_channel": ["message","button_text","send_time"],
      "youtube_shorts": ["concept","hook","script_beats","hashtags"],
      "tiktok": ["concept","hook","script_beats","hashtags"],
      "influencer_brief": ["angle","deliverables","hashtags","notes"]
    }
    prompt = f"""
채ネル別 가이드를 JSON으로 출력.
핵심 채널은 실행가능 필드, 보조 채널은 브리프 위주.
업종:{industry} / 페르소나:{persona} / 톤:{tone} / 목표:{goal}
채널:{channels}
콘셉트:{json.dumps(concepts_for_prompt, ensure_ascii=False)}
스키마:{json.dumps(schema_hint, ensure_ascii=False)}
JSON만.
""".strip()
    out={}
    try:
        import google.generativeai as genai
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        model = genai.GenerativeModel(os.getenv("GEMINI_MODEL","gemini-1.5-flash"))
        resp = model.generate_content(prompt)
        text = (resp.text or "").strip()
        m = re.search(r"\{.*\}", text, re.S)
        if m:
            raw = json.loads(m.group(0))
            out = {k: raw.get(k) for k in channels if raw.get(k)}
    except Exception:
        out = {}
    # 폴백/보강
    if "instagram" in channels and "instagram" not in out:
        out["instagram"] = {
            "format":"reels","duration":"10~12s",
            "cut_list":["외관 인서트","라떼 아트 클로즈업","디저트 플랫레이","좌석/조명 무드샷","로고/간판"],
            "copy":"오늘만큼은, 나를 위한 달콤한 휴식 🍰☕",
            "hashtags":["#성수카페","#라떼아트","#디저트맛집","#퇴근후한잔","#미식경험","#브런치","#카공카페"],
        }
    if "naver_blog" in channels and "naver_blog" not in out:
        out["naver_blog"] = {
            "title":"시험기간 카공하기 좋은 감성 카페, 디저트 추천",
            "outline":["분위기/좌석","대표 디저트","라떼 아트","이벤트 안내","위치/영업시간"],
            "cover_suggestion":"라떼 아트 클로즈업 or 디저트 플랫레이",
            "copy":"퇴근 후 보상 심리 딱 맞는 달콤한 한 잔.",
            "cta":"리뷰 작성 시 리필 쿠폰 증정"
        }
    if "kakao_channel" in channels and "kakao_channel" not in out:
        out["kakao_channel"] = {
            "message":"오늘 저녁, 나를 위한 달콤한 보상 어떠세요? 🍰",
            "button_text":"메뉴 보기",
        }
    if "naver_place" in channels and "naver_place" not in out:
        out["naver_place"] = {
            "keywords":["성수동 카페","라떼아트","디저트 맛집","카공"],
            "owner_reply_template":"방문 감사합니다 :) 더 좋은 시간 되실 수 있게 노력하겠습니다!",
            "event_copy":"리뷰 작성 시 아메리카노 리필"
        }
    # 스케줄 힌트 주입
    if schedule_hint:
        insta = out.get("instagram", {})
        if not insta.get("upload_time"):
            hint = next((r["time"] for r in schedule_hint.get("recommendations", []) if r.get("channel")=="instagram"), None)
            if hint: insta["upload_time"] = hint
        if insta: out["instagram"] = insta

        kakao = out.get("kakao_channel", {})
        if not kakao.get("send_time"):
            hint = next((r["time"] for r in schedule_hint.get("recommendations", []) if r.get("channel")=="kakao_channel"), None)
            if hint: kakao["send_time"] = hint
        if kakao: out["kakao_channel"] = kakao
    return out

# ========== 9) 최종 JSON 조립 & 파이프라인 ==========
def assemble_final_package(store_name: str, brief: dict, queries: List[str], photos6: List[dict],
                           concepts: List[dict], channel_guides: dict,
                           evidence: Optional[List[str]]=None, schedule: Optional[dict]=None) -> dict:
    return {
        "store": store_name,
        "target": f"{brief['persona']}",
        "persona": brief["persona"],
        "goal": brief.get("goal",""),
        "moodboard": [
            {"topic": p.get("alt","").strip()[:60] or "mood", "url": p["img"], "color": p.get("avg_color","#999999")}
            for p in photos6
        ],
        "queries": queries,
        "concepts": concepts,
        "channel_guides": channel_guides,
        "evidence": evidence or [],
        "schedule": schedule or {}
    }

def run_content_creator_pipeline(team2_json: dict,
                                 preferred_channels: Optional[List[str]]=None,
                                 hinted_channels: Optional[List[str]]=None,
                                 store_name: str="OO카페") -> dict:
    evidence = simple_evidence_from_team2(team2_json)
    schedule_hint = infer_schedule_from_evidence(evidence)
    brief = parse_team2_to_brief(team2_json)
    channels = decide_channels(preferred_channels, hinted_channels)
    queries, photos6, _html = build_moodboard_from_brief(brief)
    concepts = build_concepts_with_llm(brief, n=3)
    guides = build_channel_guides_with_llm(brief, concepts, channels, schedule_hint=schedule_hint)
    final = assemble_final_package(store_name, brief, queries, photos6, concepts, guides, evidence, schedule_hint)
    return final

# ========== 10) 실행 예시 (이 파일 단독 실행) ==========
if __name__ == "__main__":
    # (1) Team2 JSON — 질문에서 주신 예시 간단화
    team2_json = {
      "messages": [
        {"content": "[Team 1 종합 진단 보고서] 재방문 저하, 경쟁 강도 높음", "name": "Supervisor"},
        {"content": "STP 전략 수립이 완료되었습니다.", "name": "Supervisor"}
      ],
      "stp_strategy_document": """
      # STP 전략 보고서
      - 프리미엄 배달 시장을 공략
      - 20대 여성, 이탈률 높음 → 충성도 관리 필요
      """,
      "market_opportunities": "- 프리미엄 배달 수요 증가 ...",
      "customer_personas": "김지은 26세, 2년차 직장인, 배달 앱 VIP, 퇴근 후 미식 경험 선호",
      "targeting_proposal": "- 추천 타겟: 26세 직장인 ...",
      "positioning_statement": "- USP: '평범한 하루를 특별하게 만드는 미식 배달 경험' ..."
    }

    # (2) 브리프 생성 + 상권 형용사 주입(선택)
    brief = parse_team2_to_brief(team2_json)
    brief["market_traits"] = ["trendy","youthful"]

    print("== Brief ==")
    print(json.dumps(brief, ensure_ascii=False, indent=2))

    # (3) 무드보드 생성
    queries, photos6, html = build_moodboard_from_brief(brief)
    print("\n== Queries (6) ==")
    print(queries)
    with open("moodboard.html", "w", encoding="utf-8") as f:
        f.write(html)
    print("\n[moodboard.html] 생성 완료 — 브라우저로 열어 확인하세요.")

    # (4) 전체 패키지
    final = run_content_creator_pipeline(
        team2_json=team2_json,
        preferred_channels=["instagram","naver_blog","kakao_channel"],
        hinted_channels=[],
        store_name="OO카페"
    )
    with open("final_package.json", "w", encoding="utf-8") as f:
        json.dump(final, f, ensure_ascii=False, indent=2)
    print("[final_package.json] 생성 완료.")
    print("\n== Final Package Preview ==")
    print(json.dumps(final, ensure_ascii=False, indent=2))


== Brief ==
{
  "industry": "카페",
  "persona": "학생",
  "goal": "재방문율 상승",
  "tone": "premium",
  "season": null,
  "preferred_channels": null,
  "market_traits": [
    "trendy",
    "youthful"
  ]
}

== Queries (6) ==
['study cafe interior', 'latte art close-up', 'dessert flatlay', 'coffee bar', 'cake display', 'minimal coffee shop']

[moodboard.html] 생성 완료 — 브라우저로 열어 확인하세요.
[final_package.json] 생성 완료.

== Final Package Preview ==
{
  "store": "OO카페",
  "target": "학생",
  "persona": "학생",
  "goal": "재방문율 상승",
  "moodboard": [
    {
      "topic": "A rustic kitchen scene featuring wooden spice containers, a ",
      "url": "https://images.pexels.com/photos/6929462/pexels-photo-6929462.jpeg?auto=compress&cs=tinysrgb&fit=crop&h=1200&w=800",
      "color": "#8C6544"
    },
    {
      "topic": "A simple yet elegant glass of water on a white table in a mo",
      "url": "https://images.pexels.com/photos/7402622/pexels-photo-7402622.jpeg?auto=compress&cs=tinysrgb&fit=crop&h=1200&w=800",
     